In [1]:
%pip install gurobipy -qU

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
categories, minNutrition, maxNutrition = gp.multidict({
    'calories': [1800, 2200],
    'protein':  [91, GRB.INFINITY],
    'fat':      [0, 65],
    'sodium':   [0, 1779]})

foods, cost = gp.multidict({
    'hamburger': 2.49,
    'chicken':   2.89,
    'hot dog':   1.50,
    'fries':     1.89,
    'macaroni':  2.09,
    'pizza':     1.99,
    'salad':     2.49,
    'milk':      0.89,
    'ice cream': 1.59})

# Nutrition values for the foods
nutritionValues = {
    ('hamburger', 'calories'): 410,
    ('hamburger', 'protein'):  24,
    ('hamburger', 'fat'):      26,
    ('hamburger', 'sodium'):   730,
    ('chicken',   'calories'): 420,
    ('chicken',   'protein'):  32,
    ('chicken',   'fat'):      10,
    ('chicken',   'sodium'):   1190,
    ('hot dog',   'calories'): 560,
    ('hot dog',   'protein'):  20,
    ('hot dog',   'fat'):      32,
    ('hot dog',   'sodium'):   1800,
    ('fries',     'calories'): 380,
    ('fries',     'protein'):  4,
    ('fries',     'fat'):      19,
    ('fries',     'sodium'):   270,
    ('macaroni',  'calories'): 320,
    ('macaroni',  'protein'):  12,
    ('macaroni',  'fat'):      10,
    ('macaroni',  'sodium'):   930,
    ('pizza',     'calories'): 320,
    ('pizza',     'protein'):  15,
    ('pizza',     'fat'):      12,
    ('pizza',     'sodium'):   820,
    ('salad',     'calories'): 320,
    ('salad',     'protein'):  31,
    ('salad',     'fat'):      12,
    ('salad',     'sodium'):   1230,
    ('milk',      'calories'): 100,
    ('milk',      'protein'):  8,
    ('milk',      'fat'):      2.5,
    ('milk',      'sodium'):   125,
    ('ice cream', 'calories'): 330,
    ('ice cream', 'protein'):  8,
    ('ice cream', 'fat'):      10,
    ('ice cream', 'sodium'):   180}

In [4]:
# Model
m = gp.Model("diet")

# Create decision variables for the foods to buy
#buy = m.addVars(foods, name="buy")
buy = m.addVars(foods, lb=-1000,name = "buy")

# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
#
# buy = {}
# for f in foods:
#   buy[f] = m.addVar(name=f)

Restricted license - for non-production use only - expires 2025-11-24


In [5]:
# The objective is to minimize the costs
m.setObjective(buy.prod(cost), GRB.MINIMIZE)

# Using looping constructs, the preceding statement would be:
#
# m.setObjective(sum(buy[f]*cost[f] for f in foods), GRB.MINIMIZE)

In [6]:
# Nutrition constraints
m.addConstrs((gp.quicksum(nutritionValues[f, c] * buy[f] for f in foods)
             == [minNutrition[c], maxNutrition[c]]
             for c in categories), "_")

# Using looping constructs, the preceding statement would be:
#
# for c in categories:
#  m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods),
#             minNutrition[c], maxNutrition[c], c)

{'calories': <gurobi.Constr *Awaiting Model Update*>,
 'protein': <gurobi.Constr *Awaiting Model Update*>,
 'fat': <gurobi.Constr *Awaiting Model Update*>,
 'sodium': <gurobi.Constr *Awaiting Model Update*>}

In [7]:
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nCost: %g' % m.ObjVal)
        print('\nBuy:')
        for f in foods:
            if buy[f].X > 0.0001:
                print('%s %g' % (f, buy[f].X))
            if buy[f].X < -0.001:
                print('%s %g' % (f, buy[f].X))
    else:
        print('No solution')

In [8]:
# Solve
m.optimize()
printSolution()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4 rows, 12 columns and 39 nonzeros
Model fingerprint: 0x2ad7d20d
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+01, 2e+03]
Presolve time: 0.01s
Presolved: 4 rows, 12 columns, 39 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -7.6495312e+30   4.554321e+31   7.649531e+00      0s
       6   -2.2867863e+03   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective -2.286786275e+03

Cost: -2286.79

Buy:
hamburger 157.011
chicken 1251.59
hot dog 875.297
fries -1000
macaroni -1000
pizza -1000
salad -1000
milk -1000
ice cream 1095.73


In [9]:
m.write("diet.lp")
f = open("diet.lp","r")
contents = f.read()
print(contents)

\ Model diet
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  2.49 buy[hamburger] + 2.89 buy[chicken] + 1.5 buy[hot_dog]
   + 1.89 buy[fries] + 2.09 buy[macaroni] + 1.99 buy[pizza]
   + 2.49 buy[salad] + 0.89 buy[milk] + 1.59 buy[ice_cream]
Subject To
 calories: 410 buy[hamburger] + 420 buy[chicken] + 560 buy[hot_dog]
   + 380 buy[fries] + 320 buy[macaroni] + 320 buy[pizza] + 320 buy[salad]
   + 100 buy[milk] + 330 buy[ice_cream] + Rgcalories = 2200
 protein: 24 buy[hamburger] + 32 buy[chicken] + 20 buy[hot_dog]
   + 4 buy[fries] + 12 buy[macaroni] + 15 buy[pizza] + 31 buy[salad]
   + 8 buy[milk] + 8 buy[ice_cream] >= 91
 fat: 26 buy[hamburger] + 10 buy[chicken] + 32 buy[hot_dog] + 19 buy[fries]
   + 10 buy[macaroni] + 12 buy[pizza] + 12 buy[salad] + 2.5 buy[milk]
   + 10 buy[ice_cream] + Rgfat = 65
 sodium: 730 buy[hamburger] + 1190 buy[chicken] + 1800 buy[hot_dog]
   + 270 buy[fries] + 930 buy[macaroni] + 820 buy[pizza] + 1230 buy[salad]
   + 

In [10]:
print('\nAdding constraint: at most 6 servings of dairy')
m.addConstr(buy.sum(['milk', 'ice cream']) <= 6, "limit_dairy")

# Solve
m.optimize()
printSolution()


Adding constraint: at most 6 servings of dairy
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 12 columns and 41 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+00, 2e+03]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.2867863e+03   1.794554e+02   0.000000e+00      0s
       1   -2.2307488e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective -2.230748841e+03

Cost: -2230.75

Buy:
hamburger 141.524
chicken 1288.77
hot dog 853.017
fries -913.624
macaroni -1000
pizza -1000
salad -1000
milk -1000
ice cream 1006
